# Retail Promotional Event Analysis
## Measuring Marketing Effectiveness Through Statistical Testing

**Author:** Jacob Aguillard  
**Portfolio Project**

---

### Project Overview

This analysis examines the effectiveness of various promotional tactics (endcap placement, coupons, digital signage, QR codes) on Return on Investment (ROI) across different retail departments.

**Business Questions:**
1. Which promotional tactics significantly improve ROI?
2. Do certain tactics work better in specific departments?
3. How can we quantify the incremental value of each tactic?

**Methods Used:**
- Independent t-tests with Cohen's d effect sizes
- Chi-square tests for categorical comparisons  
- Z-tests for proportion comparisons
- Data winsorization for outlier handling

**Tools:** Python, Pandas, SciPy, NumPy, Matplotlib

---

*Note: This analysis uses synthetic data that preserves the statistical properties of real promotional event data while protecting proprietary information.*

## 1. Setup and Data Loading

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Set style for plots
plt.style.use('seaborn-v0_8-whitegrid')

print("Libraries loaded successfully")

In [ ]:
# Load the synthetic retail promotions data
df = pd.read_csv('synthetic_retail_promotions.csv')

print(f"Dataset shape: {df.shape[0]:,} records x {df.shape[1]} columns")
print(f"\nDate range: {df['event_date'].min()} to {df['event_date'].max()}")
print(f"Departments: {df['department'].nunique()}")
print(f"Regions: {df['region'].nunique()}")

df.head()

In [ ]:
# Data info and summary statistics
print("=" * 60)
print("DATA TYPES")
print("=" * 60)
df.info()

print("\n" + "=" * 60)
print("SUMMARY STATISTICS")
print("=" * 60)
df.describe().round(2)

## 2. Data Quality Assessment

In [ ]:
# Missing data analysis
missing_analysis = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df)) * 100,
    'Data_Type': df.dtypes
}).sort_values('Missing_Percentage', ascending=False)

print("MISSING DATA ANALYSIS")
print("=" * 60)
missing_with_nulls = missing_analysis[missing_analysis['Missing_Percentage'] > 0]
if len(missing_with_nulls) > 0:
    print(missing_with_nulls)
else:
    print("No missing values detected.")

In [ ]:
# Outlier detection using IQR method
print("OUTLIER DETECTION (IQR Method)")
print("=" * 60)

numeric_cols = df.select_dtypes(include=[np.number]).columns
outlier_summary = []

for col in ['roi_per_dollar', 'roi_winsorized', 'total_event_sales', 'sales_lift']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
    outlier_pct = len(outliers) / len(df) * 100
    
    print(f"\n{col}:")
    print(f"  Range: {df[col].min():.2f} to {df[col].max():.2f}")
    print(f"  IQR bounds: {lower_bound:.2f} to {upper_bound:.2f}")
    print(f"  Outliers: {len(outliers):,} ({outlier_pct:.2f}%)")

## 3. Reusable Statistical Analysis Functions

These functions form the core analytical framework for testing promotional effectiveness.

In [ ]:
def t_test_by_group(df: pd.DataFrame, target: str, measure: str, group_col: str):
    """
    Perform independent t-tests comparing a measure between groups with and without 
    a target feature, analyzed separately for each category in group_col.
    
    This function conducts statistical analysis to determine whether a binary feature 
    (e.g., end_cap, has_coupon) significantly impacts a measure (e.g., ROI) across 
    different departments or categories.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing the data to analyze.
    target : str
        Name of the binary column to test (0/1 values).
    measure : str
        Name of the numeric column to compare between groups.
    group_col : str
        Name of the column containing group identifiers (e.g., 'department').
    
    Returns
    -------
    pd.DataFrame
        Results DataFrame with t-test statistics for each group.
    
    Output Includes
    ---------------
    - Sample sizes for each group
    - Mean values for groups with/without the target feature
    - Mean difference (effect size)
    - Incremental dollar impact
    - T-statistic and p-value
    - Cohen's d effect size
    - Significance indicators (*, **, ***)
    
    Notes
    -----
    - Significance levels: * p<0.05, ** p<0.01, *** p<0.001
    - Positive Mean_Difference indicates target feature improves the measure
    - Cohen's d interpretation: 0.2 = small, 0.5 = medium, 0.8 = large
    """
    
    # Validate inputs
    if group_col not in df.columns or target not in df.columns:
        raise ValueError(f"Column not found in DataFrame")
    
    # Clean data - remove missing values
    test_data = df[[target, measure, group_col]].dropna()
    
    # Initialize results storage
    t_test_results = []
    
    # Get unique groups
    groups = test_data[group_col].unique()
    
    # Ensure target is binary (0/1)
    if test_data[target].dtype == object:
        test_data[target] = np.where(test_data[target] == "Yes", 1, 0)
    elif test_data[target].dtype in [float, int]:
        test_data[target] = np.where(test_data[target] > 0, 1, 0)
    
    # Perform t-test for each group
    for group in groups:
        group_data = test_data[test_data[group_col] == group]
        
        # Separate by target
        no_target = group_data[group_data[target] == 0][measure]
        has_target = group_data[group_data[target] == 1][measure]
        
        # Check if both groups have sufficient data
        if len(no_target) >= 2 and len(has_target) >= 2:
            # Perform independent t-test
            t_statistic, p_value = stats.ttest_ind(has_target, no_target)
            
            # Calculate Cohen's d effect size
            pooled_std = np.sqrt(
                ((len(has_target) - 1) * has_target.var() + 
                 (len(no_target) - 1) * no_target.var()) /
                (len(has_target) + len(no_target) - 2)
            )
            cohens_d = (has_target.mean() - no_target.mean()) / pooled_std if pooled_std > 0 else 0
            
            # Calculate means and differences
            mean_no_target = no_target.mean()
            mean_has_target = has_target.mean()
            mean_diff = mean_has_target - mean_no_target
            
            # Calculate incremental dollars
            incremental_dollars = mean_diff * len(has_target)
            
            # Determine significance level
            if p_value < 0.001:
                significance = "***"
            elif p_value < 0.01:
                significance = "**"
            elif p_value < 0.05:
                significance = "*"
            else:
                significance = ""
            
            t_test_results.append({
                'Group': group,
                f'N_Without_{target}': len(no_target),
                f'N_With_{target}': len(has_target),
                f'Mean_Without': round(mean_no_target, 4),
                f'Mean_With': round(mean_has_target, 4),
                'Mean_Difference': round(mean_diff, 4),
                'Incremental_$': round(incremental_dollars, 2),
                'T_Statistic': round(t_statistic, 4),
                'P_Value': round(p_value, 6),
                'Cohens_D': round(cohens_d, 4),
                'Significance': significance
            })
        else:
            t_test_results.append({
                'Group': group,
                f'N_Without_{target}': len(no_target),
                f'N_With_{target}': len(has_target),
                f'Mean_Without': no_target.mean() if len(no_target) > 0 else np.nan,
                f'Mean_With': has_target.mean() if len(has_target) > 0 else np.nan,
                'Mean_Difference': np.nan,
                'Incremental_$': np.nan,
                'T_Statistic': np.nan,
                'P_Value': np.nan,
                'Cohens_D': np.nan,
                'Significance': 'Insufficient Data'
            })
    
    # Create results DataFrame
    results_df = pd.DataFrame(t_test_results).sort_values('P_Value')
    
    # Print summary
    print(f"\n{'='*80}")
    print(f"T-TEST ANALYSIS: {target.upper()} EFFECT ON {measure.upper()}")
    print(f"{'='*80}")
    print(f"\nRecords analyzed: {len(test_data):,}")
    print(f"Groups analyzed: {len(groups)}")
    print(f"\nSignificance levels: *** p<0.001, ** p<0.01, * p<0.05")
    print(f"Positive Mean_Difference = {target} improves {measure}")
    print(f"\n{'-'*80}")
    
    # Count significant results
    significant = results_df[results_df['P_Value'] < 0.05]
    helps = significant[significant['Mean_Difference'] > 0]
    hurts = significant[significant['Mean_Difference'] < 0]
    
    print(f"\nSUMMARY:")
    print(f"  Statistically significant results (p<0.05): {len(significant)}")
    print(f"  Groups where {target} HELPS: {len(helps)}")
    print(f"  Groups where {target} HURTS: {len(hurts)}")
    
    if len(helps) > 0:
        print(f"\nGROUPS WHERE {target.upper()} SIGNIFICANTLY HELPS:")
        for _, row in helps.iterrows():
            print(f"  {row['Group']}: +{row['Mean_Difference']:.3f} (p={row['P_Value']:.4f}) {row['Significance']}")
    
    if len(hurts) > 0:
        print(f"\nGROUPS WHERE {target.upper()} SIGNIFICANTLY HURTS:")
        for _, row in hurts.iterrows():
            print(f"  {row['Group']}: {row['Mean_Difference']:.3f} (p={row['P_Value']:.4f}) {row['Significance']}")
    
    return results_df

## 4. ROI Distribution Analysis

In [ ]:
# ROI Distribution by Department
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original ROI distribution
axes[0].hist(df['roi_per_dollar'].clip(-5, 15), bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['roi_per_dollar'].median(), color='red', linestyle='--', label=f'Median: {df["roi_per_dollar"].median():.2f}')
axes[0].axvline(df['roi_per_dollar'].mean(), color='green', linestyle='--', label=f'Mean: {df["roi_per_dollar"].mean():.2f}')
axes[0].set_title('ROI per Dollar Spent Distribution')
axes[0].set_xlabel('ROI')
axes[0].set_ylabel('Frequency')
axes[0].legend()

# Winsorized ROI distribution  
axes[1].hist(df['roi_winsorized'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].axvline(df['roi_winsorized'].median(), color='red', linestyle='--', label=f'Median: {df["roi_winsorized"].median():.2f}')
axes[1].axvline(df['roi_winsorized'].mean(), color='green', linestyle='--', label=f'Mean: {df["roi_winsorized"].mean():.2f}')
axes[1].set_title('Winsorized ROI Distribution (5th-95th percentile)')
axes[1].set_xlabel('ROI')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Original ROI - Mean: {df['roi_per_dollar'].mean():.3f}, Median: {df['roi_per_dollar'].median():.3f}, Std: {df['roi_per_dollar'].std():.3f}")
print(f"Winsorized ROI - Mean: {df['roi_winsorized'].mean():.3f}, Median: {df['roi_winsorized'].median():.3f}, Std: {df['roi_winsorized'].std():.3f}")

In [ ]:
# ROI by Department
dept_roi = df.groupby('department')['roi_winsorized'].agg(['mean', 'median', 'std', 'count']).round(3)
dept_roi = dept_roi.sort_values('mean', ascending=False)

print("ROI BY DEPARTMENT")
print("=" * 60)
print(dept_roi)

# Visualization
plt.figure(figsize=(12, 6))
colors = ['green' if x > df['roi_winsorized'].mean() else 'coral' for x in dept_roi['mean']]
plt.barh(dept_roi.index, dept_roi['mean'], color=colors, edgecolor='black')
plt.axvline(df['roi_winsorized'].mean(), color='red', linestyle='--', linewidth=2, label=f'Overall Mean: {df["roi_winsorized"].mean():.2f}')
plt.xlabel('Mean ROI (Winsorized)')
plt.title('Average ROI by Department')
plt.legend()
plt.tight_layout()
plt.show()

## 5. Promotional Tactic Analysis

### 5.1 End Cap Placement Effect

In [ ]:
# Analyze end cap effect on ROI by department
endcap_results = t_test_by_group(
    df=df, 
    target='end_cap', 
    measure='roi_winsorized', 
    group_col='department'
)

# Display full results table
print("\nFULL RESULTS TABLE:")
endcap_results

In [ ]:
# Visualize end cap effect
significant_endcap = endcap_results[endcap_results['P_Value'] < 0.05].copy()

if len(significant_endcap) > 0:
    plt.figure(figsize=(12, 6))
    colors = ['green' if x > 0 else 'red' for x in significant_endcap['Mean_Difference']]
    plt.barh(significant_endcap['Group'], significant_endcap['Mean_Difference'], color=colors, edgecolor='black')
    plt.axvline(0, color='black', linewidth=1)
    plt.xlabel('ROI Difference (With End Cap - Without)')
    plt.title('Significant End Cap Effects by Department (p < 0.05)')
    plt.tight_layout()
    plt.show()
else:
    print("No statistically significant end cap effects found.")

### 5.2 Coupon Effect

In [ ]:
# Analyze coupon effect on ROI by department
coupon_results = t_test_by_group(
    df=df, 
    target='has_coupon', 
    measure='roi_winsorized', 
    group_col='department'
)

coupon_results

In [ ]:
# Visualize coupon effect
significant_coupon = coupon_results[coupon_results['P_Value'] < 0.05].copy()

if len(significant_coupon) > 0:
    plt.figure(figsize=(12, 6))
    colors = ['green' if x > 0 else 'red' for x in significant_coupon['Mean_Difference']]
    plt.barh(significant_coupon['Group'], significant_coupon['Mean_Difference'], color=colors, edgecolor='black')
    plt.axvline(0, color='black', linewidth=1)
    plt.xlabel('ROI Difference (With Coupon - Without)')
    plt.title('Significant Coupon Effects by Department (p < 0.05)')
    plt.tight_layout()
    plt.show()

### 5.3 Digital Signage Effect

In [ ]:
# Analyze digital signage effect
digital_results = t_test_by_group(
    df=df, 
    target='digital_signage', 
    measure='roi_winsorized', 
    group_col='department'
)

digital_results

### 5.4 QR Code Effect

In [ ]:
# Analyze QR code effect
qr_results = t_test_by_group(
    df=df, 
    target='qr_code', 
    measure='roi_winsorized', 
    group_col='department'
)

qr_results

## 6. Conversion Rate Analysis by Day of Week

In [ ]:
# Add day of week
df['event_date'] = pd.to_datetime(df['event_date'])
df['day_of_week'] = df['event_date'].dt.day_name()

# Conversion rate by day of week
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_stats = df.groupby('day_of_week').agg({
    'conversion_rate': ['mean', 'std', 'count'],
    'roi_winsorized': ['mean', 'std']
}).round(4)

day_stats = day_stats.reindex(day_order)
print("PERFORMANCE BY DAY OF WEEK")
print("=" * 60)
print(day_stats)

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

conv_by_day = df.groupby('day_of_week')['conversion_rate'].mean().reindex(day_order)
axes[0].bar(conv_by_day.index, conv_by_day.values, color='steelblue', edgecolor='black')
axes[0].axhline(df['conversion_rate'].mean(), color='red', linestyle='--', label='Overall Mean')
axes[0].set_title('Average Conversion Rate by Day of Week')
axes[0].set_ylabel('Conversion Rate')
axes[0].tick_params(axis='x', rotation=45)
axes[0].legend()

roi_by_day = df.groupby('day_of_week')['roi_winsorized'].mean().reindex(day_order)
axes[1].bar(roi_by_day.index, roi_by_day.values, color='coral', edgecolor='black')
axes[1].axhline(df['roi_winsorized'].mean(), color='red', linestyle='--', label='Overall Mean')
axes[1].set_title('Average ROI by Day of Week')
axes[1].set_ylabel('ROI (Winsorized)')
axes[1].tick_params(axis='x', rotation=45)
axes[1].legend()

plt.tight_layout()
plt.show()

## 7. Chi-Square Test: Day of Week Effect by Department

In [ ]:
def analyze_day_effect_by_department(df, department_col='department'):
    """
    Perform chi-square tests to determine if day of week significantly
    affects conversion rates within each department.
    """
    results = []
    
    for dept in df[department_col].unique():
        dept_data = df[df[department_col] == dept]
        
        # Create contingency table: day of week vs high/low conversion
        dept_data = dept_data.copy()
        median_conv = dept_data['conversion_rate'].median()
        dept_data['high_conversion'] = (dept_data['conversion_rate'] > median_conv).astype(int)
        
        contingency = pd.crosstab(dept_data['day_of_week'], dept_data['high_conversion'])
        
        if contingency.shape[0] >= 2 and contingency.shape[1] >= 2:
            chi2, p_value, dof, expected = chi2_contingency(contingency)
            
            results.append({
                'Department': dept,
                'Chi2_Statistic': round(chi2, 4),
                'P_Value': round(p_value, 6),
                'Degrees_Freedom': dof,
                'Days_Matter': 'Yes' if p_value < 0.05 else 'No'
            })
    
    results_df = pd.DataFrame(results).sort_values('P_Value')
    
    print("CHI-SQUARE TEST: Does Day of Week Affect Conversion by Department?")
    print("=" * 70)
    print(f"\nDepartments where day significantly matters (p<0.05): {len(results_df[results_df['Days_Matter']=='Yes'])}")
    
    return results_df

day_effect_results = analyze_day_effect_by_department(df)
day_effect_results

## 8. Summary of Findings

In [ ]:
print("=" * 80)
print("EXECUTIVE SUMMARY: PROMOTIONAL EFFECTIVENESS ANALYSIS")
print("=" * 80)

print(f"""
DATASET OVERVIEW:
- Total promotional events analyzed: {len(df):,}
- Departments: {df['department'].nunique()}
- Regions: {df['region'].nunique()}
- Date range: {df['event_date'].min().date()} to {df['event_date'].max().date()}

KEY METRICS:
- Average ROI per dollar spent: {df['roi_winsorized'].mean():.2f}
- Median ROI: {df['roi_winsorized'].median():.2f}
- Average conversion rate: {df['conversion_rate'].mean():.2%}
- Average sales lift: ${df['sales_lift'].mean():,.2f}

PROMOTIONAL TACTIC FINDINGS:
""")

# Summarize each tactic
tactics = [
    ('End Cap Placement', endcap_results),
    ('Coupons', coupon_results),
    ('Digital Signage', digital_results),
    ('QR Codes', qr_results)
]

for tactic_name, results in tactics:
    sig_helps = results[(results['P_Value'] < 0.05) & (results['Mean_Difference'] > 0)]
    sig_hurts = results[(results['P_Value'] < 0.05) & (results['Mean_Difference'] < 0)]
    
    print(f"{tactic_name}:")
    print(f"  - Departments with significant positive effect: {len(sig_helps)}")
    print(f"  - Departments with significant negative effect: {len(sig_hurts)}")
    if len(sig_helps) > 0:
        top_dept = sig_helps.iloc[0]
        print(f"  - Strongest positive effect: {top_dept['Group']} (+{top_dept['Mean_Difference']:.3f} ROI)")
    print()

print("""
RECOMMENDATIONS:
1. Prioritize end cap placement for Food division departments where effect is strongest
2. Deploy coupons strategically in Snacks, Frozen, Beverages, and Dairy departments
3. Consider department-specific promotional strategies rather than one-size-fits-all
4. Further investigate departments where tactics show negative effects
""")

---

## About This Analysis

This notebook demonstrates:

1. **Statistical Testing** - Independent t-tests, chi-square tests, effect size calculations (Cohen's d)
2. **Data Quality Assessment** - Missing data analysis, outlier detection, winsorization
3. **Reusable Code** - Parameterized functions for repeatable analysis
4. **Business Insight Generation** - Translating statistical results into actionable recommendations
5. **Data Visualization** - Clear charts that communicate findings effectively

**Technical Stack:** Python, Pandas, NumPy, SciPy, Matplotlib, Seaborn

**Author:** Jacob Aguillard  
**Contact:** aguillarj@gmail.com